In [7]:
import cv2
import numpy as np
from keras.models import load_model
from keras.applications.mobilenet_v2 import preprocess_input

In [2]:
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
mask_model = load_model('ModelWeights/Mobilenet_GithubMobile.h5')

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


In [ ]:
#live video

bw = False 
mobile = True 
cap = cv2.VideoCapture(0)
dim = (224,224)
mask_dict = {0: 'No Mask', 1: 'Mask'}
mask_dict_color = {0: (0,0,255), 1: (0,255,0)}

while True: 
    ret, frame = cap.read()
    clone = frame.copy()
    bboxes = classifier.detectMultiScale(clone)
    for i in bboxes: 
        x, y, width, height = i[0], i[1], i[2], i[3]
        x2, y2 = x+ width, y+height
        roi = clone[y:y2, x:x2]
        roi = cv2.resize(roi, dim, interpolation = cv2.INTER_CUBIC)
        if bw == True: 
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            roi = roi.reshape(roi.shape[0], roi.shape[1],1)
        if mobile == True: 
            roi = preprocess_input(roi)
        roi = roi.reshape(1, roi.shape[0], roi.shape[1], roi.shape[2])
        mask_predict = mask_model.predict(roi)[0]
        predict_idx = np.argmax(mask_predict)
        predict_conf = f'{round(np.max(mask_predict))*100}%'
#         print(mask_predict)
        cv2.putText(clone, f'{mask_dict[predict_idx]}: {predict_conf}', (x, y+15), cv2.FONT_HERSHEY_SIMPLEX, .5, mask_dict_color[predict_idx], 2)
        cv2.rectangle(clone, (x,y), (x2,y2), mask_dict_color[predict_idx], 1)
        
        
    
    
    cv2.imshow('LIVE', clone)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'): 
        break 
cap.release()

cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('../multiple.png')

In [ ]:
bboxes = classifier.detectMultiScale(img)

In [ ]:
clone = img.copy()
for i in bboxes: 
    x, y, width, height = i[0], i[1], i[2], i[3]
    x2, y2 = x+ width, y+height
    cv2.rectangle(clone, (x,y), (x2,y2), (0,0,255), 1)
    
    
cv2.imshow('test', clone)
cv2.waitKey(0)
cv2.destroyAllWindows()
    